In [1]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import datetime
import csv
import os
from sklearn.metrics import r2_score, mean_squared_error

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from modules.prediction import load_all_data
from modules.prediction import precrime_train_test_split
from modules.prediction import load_splits
from modules.prediction import create_all_splits
from modules.prediction import sample_model
from modules.poly_ridge import poly_ridge_model
from modules.fancy_time_series import fancy_time_series_model
from modules.eval_model import eval_predictions
from modules.prediction import create_test_period

In [3]:
crime_data = load_all_data()

/Users/jsamet/anaconda2/envs/py3k/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
split_future = create_test_period(datetime.date(2017,10,1),datetime.date(2018,9,30))

In [5]:
train_test_data = create_all_splits(crime_data, {'future': split_future})

In [6]:
X_train_future, _, y_train_future, _ = train_test_data['future']

In [7]:
average_weather = X_train_future.groupby([
    'COMPLAINT_MONTH',
    'COMPLAINT_DAY'
])[[
    'temperature',
    'precipIntensity'
]].mean().reset_index()

precinct_codes = pd.DataFrame(X_train_future['ADDR_PCT_CD'].unique(), columns=['ADDR_PCT_CD'])
precinct_codes['key'] = 1

gimme_a_datetime = split_future.copy()
gimme_a_datetime.columns = [c[5:].lower() for c in gimme_a_datetime.columns]
gimme_a_datetime['dayofweek'] = pd.to_datetime(gimme_a_datetime[['year', 'month', 'day']]).dt.dayofweek
gimme_a_datetime.columns = ['COMPLAINT_{0}'.format(c.upper()) for c in gimme_a_datetime.columns]
gimme_a_datetime['key'] = 1

dates_and_precincts = pd.merge(gimme_a_datetime, precinct_codes, how='outer')
dates_and_precincts.drop('key', axis=1, inplace=True)


In [8]:
X_test_future = pd.merge(dates_and_precincts, average_weather)
y_poly_future = poly_ridge_model(X_train_future, y_train_future, X_test_future)

In [9]:
crime_data_with_poly_future = pd.concat(
    [crime_data, pd.merge(X_test_future, y_poly_future)],
    ignore_index=True
)
train_test_data_with_poly_future = create_all_splits(crime_data_with_poly_future, {'ts_future': split_future})
X_train_ts_future, X_test_ts_future, y_train_ts_future, y_test_ts_future = train_test_data_with_poly_future['ts_future']
y_ts_future = fancy_time_series_model(X_train_ts_future, y_train_ts_future, X_test_ts_future, y_test_ts_future).reset_index().drop('index', axis=1)

In [10]:
y_hybrid_future = (y_poly_future + y_ts_future)/2

In [12]:
key_fields = [
    'COMPLAINT_YEAR',
    'COMPLAINT_MONTH',
    'COMPLAINT_DAY',
    'COMPLAINT_HOURGROUP',
    'ADDR_PCT_CD',
]
offense_types = [x for x in y_train_future.select_dtypes(exclude=['object']).columns]

In [13]:
y_hybrid_future.groupby([
    'COMPLAINT_MONTH',
    'ADDR_PCT_CD'
])[offense_types].sum()

Homicide      Rape    Robbery  FelonyAssault  \
COMPLAINT_MONTH ADDR_PCT_CD                                                 
1.0             1.0          0.072230  2.100566   4.871484       7.862098   
                5.0          0.027475  1.419239   5.919904      15.654493   
                6.0          0.023864  1.452340   9.285432      10.476005   
                7.0          0.029462  1.535719   8.617750      12.163267   
                9.0          0.044028  1.840533   9.772930      13.518679   
                10.0         0.028449  2.045123   6.379834      10.398740   
                13.0         0.161874  3.322637   9.328179      13.837298   
                14.0         0.209277  4.327782  12.514388      17.612233   
                17.0         0.069861  1.339258   4.461581       6.618433   
                18.0         0.030496  3.751515   9.031067      13.647970   
                19.0         0.025606  2.139062   8.568891      10.219447   
                20.0         0.044477  1.128501   4.983057       5.978508   
                22.0         0.014533  0.496937   1.314037       2.411473   
                23.0         0.217509  2.514077  14.818112      24.144456   
                24.0         0.085339  1.864990  10.300719      10.988389   
                25.0         0.158173  2.984062  14.822988      26.100192   
                26.0         0.068634  1.266239   8.354072       8.709333   
                28.0         0.213345  1.911272  15.320322      18.450812   
                30.0         0.112800  2.147981  13.314081      14.980092   
                32.0         0.442515  3.402112  17.025166      27.933162   
                33.0         0.065916  1.708567  10.651941      16.523795   
                34.0         0.144081  2.292080  16.512102      21.587448   
                40.0         0.639803  3.796465  36.226899      52.328226   
                41.0         0.175059  3.116463  14.990443      45.420306   
                42.0         0.613545  3.564091  21.490263      34.229814   
                43.0         0.687555  4.885708  37.531116      49.823481   
                44.0         0.813910  4.262988  33.861349      53.126010   
                45.0         0.159949  1.457299  10.258773      15.778356   
                46.0         0.729391  4.028204  32.447752      44.962442   
                47.0         0.982508  4.546599  32.980109      50.552959   
...                               ...       ...        ...            ...   
12.0            76.0         0.050163  0.633277   5.731721       8.428558   
                77.0         0.671879  2.725133  22.412196      28.251453   
                78.0         0.032347  0.884320   9.116238       7.693743   
                79.0         0.571787  3.070042  29.350990      34.027492   
                81.0         0.691607  2.625493  23.112031      26.673274   
                83.0         0.372277  3.993401  29.346838      29.750328   
                84.0         0.174764  0.939706  11.351038      13.282404   
                88.0         0.181813  1.081208  12.927032      14.903144   
                90.0         0.362211  2.601493  23.039784      21.618517   
                94.0         0.131021  0.808943   7.491904       8.085642   
                100.0        0.098029  0.702736   5.259532      11.540259   
                101.0        0.314591  1.693297  10.002470      22.674368   
                102.0        0.295761  2.204824  17.670351      20.093665   
                103.0        0.494606  3.494905  32.674312      34.488748   
                104.0        0.122117  1.795002  16.359820      17.795634   
                105.0        0.436605  3.019455  22.888119      29.543605   
                106.0        0.369511  1.451624  20.545831      19.545072   
                107.0        0.066181  1.667307  16.620262      11.083998   
                108.0        0.060135  1.859653  10.441707      12.831729   
                109.0    

In [21]:
crime_data[crime_data['COMPLAINT_YEAR'] == 2016].groupby([
    'COMPLAINT_MONTH',
    'ADDR_PCT_CD'
])[offense_types].sum() - \
y_hybrid_future.groupby([
    'COMPLAINT_MONTH',
    'ADDR_PCT_CD'
])[offense_types].sum()

Homicide      Rape    Robbery  FelonyAssault  \
COMPLAINT_MONTH ADDR_PCT_CD                                                 
1               1           -0.072230 -0.100566  -0.871484      -4.862098   
                5           -0.027475 -0.419239   1.080096      11.345507   
                6           -0.023864 -0.452340   7.714568       3.523995   
                7           -0.029462  0.464281   0.382250       1.836733   
                9           -0.044028 -0.840533   1.227070      -1.518679   
                10          -0.028449 -1.045123  -1.379834      -2.398740   
                13          -0.161874 -1.322637   3.671821      -1.837298   
                14          -0.209277  0.672218   1.485612       3.387767   
                17          -0.069861 -1.339258   2.538419       1.381567   
                18          -0.030496 -2.751515  10.968933       2.352030   
                19          -0.025606 -0.139062   2.431109      -1.219447   
                20          -0.044477  2.871499   1.016943       0.021492   
                22          -0.014533 -0.496937  -1.314037      -2.411473   
                23          -0.217509  1.485923   2.181888      14.855544   
                24          -0.085339 -1.864990   1.699281      -0.988389   
                25           0.841827  0.015938  -0.822988      -9.100192   
                26          -0.068634  0.733761   7.645928       0.290667   
                28          -0.213345 -1.911272  -1.320322       3.549188   
                30          -0.112800  0.852019   1.685919       2.019908   
                32          -0.442515 -2.402112   0.974834       3.066838   
                33          -0.065916 -1.708567  -2.651941      -3.523795   
                34          -0.144081 -1.292080   1.487898       4.412552   
                40           0.360197  0.203535  11.773101      -2.328226   
                41          -0.175059 -1.116463   5.009557      11.579694   
                42          -0.613545 -2.564091   0.509737       7.770186   
                43          -0.687555 -0.885708  21.468884      -3.823481   
                44          -0.813910 -2.262988   9.138651      -1.126010   
                45          -0.159949  0.542701   4.741227       5.221644   
                46          -0.729391 -1.028204   6.552248     -15.962442   
                47           0.017492 -0.546599  12.019891       2.447041   
...                               ...       ...        ...            ...   
12              76          -0.050163 -0.633277  -2.731721      -4.428558   
                77          -0.671879  0.274867  -0.412196      -3.251453   
                78          -0.032347 -0.884320  -1.116238      -1.693743   
                79          -0.571787 -2.070042   7.649010      -4.027492   
                81          -0.691607 -0.625493  -3.112031     -10.673274   
                83          -0.372277 -1.993401  -5.346838      -4.750328   
                84          -0.174764 -0.939706  -3.351038      -0.282404   
                88          -0.181813 -1.081208   0.072968       4.096856   
                90           0.637789  1.398507  -6.039784      -2.618517   
                94          -0.131021 -0.808943  -2.491904      -2.085642   
                100         -0.098029 -0.702736  -1.259532      -2.540259   
                101         -0.314591  0.306703   6.997530      -4.674368   
                102         -0.295761 -1.204824  -1.670351      -1.093665   
                103         -0.494606  0.505095  -6.674312       1.511252   
                104         -0.122117  0.204998  -0.359820      -0.795634   
                105         -0.436605 -0.019455  -4.888119      -6.543605   
                106          1.630489  0.548376  -5.545831      -2.545072   
                107         -0.066181 -1.667307   0.379738      -3.083998   
                108         -0.060135  2.140347  -1.441707      -0.831729   
                109      